<a href="https://colab.research.google.com/github/jeniferss/MCZA017-13_PLN/blob/main/2025_q3_pln_projeto_final_first_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2025-Q3]**
Prof. Alexandre Donizeti Alves

### **PROJETO PRÁTICO** [LangChain + Grandes Modelos de Linguagem]


O **PROJETO PRÁTICO** deve ser feito utilizando o **Google Colab** com uma conta sua vinculada ao Gmail. O link do seu notebook armazenado no Google Drive e o link de um repositório no GitHub devem ser enviados usando o seguinte formulário:

> https://forms.gle/D4gLqP1iGgyn2hbH8


**IMPORTANTE**: A submissão deve ser feita até o dia **07/12 (domingo)** APENAS POR UM INTEGRANTE DA EQUIPE, até às 23h59. Por favor, lembre-se de dar permissão de ACESSO IRRESTRITO para o professor da disciplina.

### **EQUIPE**

---

**POR FAVOR, PREENCHER OS INTEGRANDES DA SUA EQUIPE:**


**Integrante 01:**

`JENIFER SOARES SOUZA           11202020219`

**Integrante 02:**

`JUAN PABLO MORENO OLIVEIRA     11202131494`

### **GRANDE MODELO DE LINGUAGEM (*Large Language Model - LLM*)**

---

Cada equipe deve selecionar um Grande Modelo de Linguagem (*Large Language Model - LMM*).



Por favor, informe os dados do LLM selecionada:

>


**LLM**: GPT-4.1 mini ou gemini-2.5-flash

>

**Link para a documentação oficial**: https://openai.com/index/gpt-4-1/ ou https://docs.cloud.google.com/vertex-ai/generative-ai/docs/models/gemini/2-5-flash?hl=pt-br



### **API (Opcional)**
---

Por favor, informe os dados da API selecionada:

**API**:Gutendex

**Site oficial**:https://gutendex.com/

**Link para a documentação oficial**:https://gutendex.com/ (na homepage)






### **DESCRIÇÃO**
---

Implementar um `notebook` no `Google Colab` que faça uso do framework **`LangChain`** (obrigatório) e de um **LLM** aplicando, no mínimo, DUAS técnicas de PLN. As técnicas podem ser aplicada em qualquer córpus obtido a partir de uma **API** ou a partir de uma página Web.

O **LLM** e a **API** selecionados devem ser informados na seguinte planilha:

> https://docs.google.com/spreadsheets/d/1iIUZcwnywO7RuF6VEJ8Rx9NDT1cwteyvsnkhYr0NWtU/edit?usp=sharing

>
As seguintes técnicas de PLN podem ser usadas:

*   Correção Gramatical
*   Classificação de Textos
*   Análise de Sentimentos
*   Detecção de Emoções
*   Extração de Palavras-chave
*   Tradução de Textos
*   Sumarização de Textos
*   Similaridade de Textos
*   Reconhecimento de Entidades Nomeadas
*   Sistemas de Perguntas e Respostas
>

**IMPORTANTE:** É obrigatório usar o e-mail da UFABC.


### **CRITÉRIOS DE AVALIAÇÃO**
---


Serão considerados como critérios de avaliação os seguintes pontos:

* Uso do framework **`LangChain`**.

* Escolha e uso de um **LLM**.

* Escolha e uso de uma **API** ou **Página Web**.

* Projeto disponível no Github.

* Apresentação (5 a 10 minutos).

* Criatividade no uso do framework **`LangChain`** em conjunto com o **LLM** e a **API**.




**IMPORTANTE**: todo o código do notebook deve ser executado. Código sem execução não será considerado.

### **IMPLEMENTAÇÃO**
---

Neste trabalho, utilizamos como fonte de dados o [Projeto Gutenberg](https://www.gutenberg.org/about/), que disponibiliza obras literárias em domínio público. A partir desse acervo, selecionamos quatro livros brasileiros de interesse para análise — *Dom Casmurro*, *O Cortiço* e *Quincas Borba*. Para integrá-los ao pipeline, realizamos o mapeamento manual de seus respectivos *IDs* no catálogo e utilizamos a API [Gutendex](https://gutendex.com/) para recuperar metadados essenciais, incluindo as URLs que fornecem o texto completo em formato **.txt**.

#### Instalação de bibliotecas

In [ ]:
!pip install -U "langchain[google-genai]"

#### Importação de bibliotecas

In [ ]:
import csv
import json
import json
import os
import pandas as pd
import pandas as pd
import re
import requests
import spacy
import unicodedata
from google.colab import userdata
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, HttpUrl
from spacy.lang.en.stop_words import STOP_WORDS as EN_STOPWORDS
from time import sleep
from typing import List, Optional, Dict, Set

#### Definição dos modelos de dados

In [ ]:
class Person(BaseModel):
    name: str
    birth_year: Optional[int] = None
    death_year: Optional[int] = None


class BookMetadataInput(BaseModel):
    id: int
    title: str
    authors: List[Person]
    summaries: List[str] = []
    editors: List[Person] = []
    translators: List[Person] = []
    subjects: List[str] = []
    bookshelves: List[str] = []
    languages: List[str] = []
    copyright: Optional[bool] = None
    media_type: Optional[str] = None
    formats: Dict[str, HttpUrl]
    download_count: int


class BookMetadataOutput(BaseModel):
    title: str
    authors: List[Person]
    summaries: List[str] = []
    subjects: List[str] = []
    characters: List[str] = []
    languages: List[str] = []
    copyright: Optional[bool] = None
    media_type: Optional[str] = None
    text_url: HttpUrl


class CharacterLLMInput(BaseModel):
    book: BookMetadataOutput
    character_name: str
    snippets: Dict[int, str]

#### Definição de funções auxiliares

In [ ]:
def _normalize_name(name: str) -> str:
    name = name.strip().strip('"').strip("'").strip()
    name = re.sub(r"[’'´`]+s$", "", name)
    return name


def _name_tokens(name: str) -> Set[str]:
    return {t for t in re.split(r"\s+", name) if t}


def _author_token_set(authors) -> Set[str]:
    tokens = set()
    for a in authors:
        norm = _normalize_name(a.name.replace(",", " "))
        tokens.update(_name_tokens(norm))
    return tokens


def _is_stop_candidate(token_text: str) -> bool:
    if not token_text:
        return True
    if token_text.lower() in EN_STOPWORDS:
        return True
    if len(token_text) <= 2:
        return True
    return False


def _strip_accents(s: str) -> str:
    nfkd = unicodedata.normalize("NFD", s)
    return "".join(ch for ch in nfkd if unicodedata.category(ch) != "Mn")


def _split_paragraphs(text: str) -> List[str]:
    raw_paragraphs = re.split(r"\n\s*\n", text)
    return [p.strip() for p in raw_paragraphs if p.strip()]


def extract_book_metadata(book: int) -> BookMetadataInput:
    response = requests.get(f"https://gutendex.com/books/{book}")
    response.raise_for_status()

    gutendex = BookMetadataInput.model_validate(response.json())
    return gutendex


def extract_character_names(book) -> List[str]:
    text = " ".join(book.summaries or [])
    doc = nlp(text)

    author_tokens = _author_token_set(book.authors)
    raw_candidates: Set[str] = set()

    for ent in doc.ents:
        if ent.label_ != "PERSON":
            continue

        name = _normalize_name(ent.text)
        if not name:
            continue

        tokens = _name_tokens(name)
        if not tokens:
            continue

        if tokens.issubset(author_tokens):
            continue

        raw_candidates.add(name)

    return sorted(raw_candidates)


def enrich_books_with_characters(book: BookMetadataInput) -> BookMetadataOutput:
    return BookMetadataOutput(
        authors=book.authors,
        title=book.title,
        summaries=book.summaries,
        subjects=book.subjects,
        characters=extract_character_names(book),
        text_url=book.formats['text/plain; charset=us-ascii']
    )


def find_relevant_paragraphs(
        paragraphs: List[str],
        character_name: str,
        window: int = 1,
        min_length: int = 80,
) -> List[str]:
    max_snippets_in_prompt = 20

    name_norm = _strip_accents(character_name).lower()
    pattern = re.compile(rf"\b{re.escape(name_norm)}\b")

    hit_indices: List[int] = []
    for i, p in enumerate(paragraphs):
        p_norm = _strip_accents(p).lower()
        if pattern.search(p_norm):
            hit_indices.append(i)

    if not hit_indices:
        return {}

    selected_indices: List[int] = []
    seen: Set[int] = set()

    for idx in hit_indices:
        start = max(0, idx - window)
        end = min(len(paragraphs), idx + window + 1)

        for j in range(start, end):
            if j not in seen:
                seen.add(j)
                selected_indices.append(j)

    selected_indices.sort()
    return {
        i: paragraphs[i].replace("\r\n", "\n").replace("\r", "")
        for i in selected_indices[:max_snippets_in_prompt] if len(paragraphs[i].strip()) >= min_length
    }


def build_character_llm_input(
        book_output: BookMetadataOutput,
        character_name: str,
        window: int = 1
) -> CharacterLLMInput:
    response = requests.get(book_output.text_url)
    response.raise_for_status()

    paragraphs = _split_paragraphs(response.text)
    snippets = find_relevant_paragraphs(
        paragraphs,
        character_name=character_name,
        window=window
    )

    return CharacterLLMInput(
        book=book_output,
        character_name=character_name,
        snippets=snippets
    )


def build_prompt_from_context(ctx: CharacterLLMInput) -> str:
    data = json.dumps({
        "metadados_livro": {
            "titulo": ctx.book.title,
            "autores": [a.model_dump() for a in ctx.book.authors],
            "assuntos": ctx.book.subjects,
            "personagens_detectados_no_resumo": ctx.book.characters
        },
        "personagem_alvo": {
            "nome": ctx.character_name
        },
        "trechos_texto": ctx.snippets
    }, ensure_ascii=False)
    return data


def process_book_characters(book: BookMetadataOutput) -> List[dict]:
    results: List[dict] = []

    for character_name in book.characters:
        try:
            if not character_name.strip:
                continue

            print(f"\n\nProcessando {character_name} para o livro {book.title}")
            ctx = build_character_llm_input(
                book_output=book,
                character_name=character_name,
                window=1,
            )

            if not book.characters:
                print(f"Nenhum personagem encontrado para o livro {book.title}")
                continue

            if not ctx.snippets:
                print(f"Nenhum parágrafo encontrado personagem de nome {character_name} encontrado para o livro {book.title}")

            prompt_text = build_prompt_from_context(ctx)
            llm_output = trait_chain.invoke({"texto": prompt_text})

            results.append(
                {
                    "character_name": character_name,
                    "character_context": ctx,
                    "llm_output": llm_output,
                }
            )

            build_row_from_result(ctx, llm_output)
            sleep(1)

        except Exception as error:
            print("Falha ao extrair dados do personagem", str(error))

    return results


def build_row_from_result(ctx: CharacterLLMInput, llm_output: str) -> List[dict]:
    book = ctx.book

    authors_json = json.dumps([a.model_dump() for a in book.authors], ensure_ascii=False)
    summaries_json = json.dumps(book.summaries, ensure_ascii=False)
    subjects_json = json.dumps(book.subjects, ensure_ascii=False)
    characters_json = json.dumps(book.characters, ensure_ascii=False)

    cleaned = re.sub(r"^```(\w+)?", "", llm_output.strip())
    cleaned = re.sub(r"```$", "", cleaned).strip()

    inferences = json.loads(cleaned)["inferences"]
    resume = json.loads(cleaned)["resume"]

    results = []
    for inference in inferences:
        row = {
            "title": book.title,
            "authors": authors_json,
            "summaries": summaries_json,
            "subjects": subjects_json,
            "characters": characters_json,
            "text_url": book.text_url,
            "character_name": ctx.character_name,
            "character_resume": resume,
            "snippet": ctx.snippets.get(int(inference["id"])),
            "traits": json.dumps(inference["traits"], ensure_ascii=False),
        }

        append_row_to_csv(row)
        results.append(row)

    return results


def append_row_to_csv(row: dict, path: str = "characters_dataset.csv") -> None:
    file_exists = os.path.exists(path)

    with open(path, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=row.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)

#### Configuração para a pipeline



In [ ]:
nlp = spacy.load("en_core_web_sm")

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
print(f"\n\nGOOGLE_API_KEY está vazia." if GOOGLE_API_KEY is None else "")

llm_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.0,
)

trait_prompt = PromptTemplate(
    input_variables=["texto"],
    template=("""
        Você é um analista especializado em perfis de personagens literários. Com base EXCLUSIVAMENTE nos trechos fornecidos abaixo, produza um JSON com inferências de personalidade. Regras obrigatórias:
        - A saída deve conter apenas JSON. Não inclua explicações.
        - O JSON final deve ter exatamente as chaves:
        - "inferences": lista de objetos.
        - "resume": um resumo com NO MÁXIMO 200 caracteres EM PORTUGUÊS BARSILEIRO sobre quem é o personagem com base EM TODOS OS TRECHOS enviado no contexto como "trechos_texto".
        - Cada objeto dentro de "inferences" deve seguir esta estrutura:
        - "id": o mesmo id recebido na entrada, correspondente ao trecho analisado.
        - "traits": lista de adjetivos que descrevem a personalidade detectada NO TRECHO.
        - Você deve retirar apenas traits claramente justificáveis pelo trecho.
        - Se não houver traits relevantes, NÃO inclua esse item na lista "inferences".
        - Todos os adjetivos devem ser:
        - em PORTUGUÊS BRASILEIRO,
        - no gênero masculino,
        - escolhidos APENAS desta lista:
        astuto, inteligente, rabugento, sensível, misterioso, impulsivo, determinado, melancólico, orgulhoso, cínico, ingênuo, corajoso, leal, ambicioso, manipulador, compulsivo, vaidoso, inseguro, ciumento, observador, tímido, protetor, teimoso, irônico, frio, calculista, curioso, sonhador, dramático, solitário, pessimista, temperamental, independente, romântico, carismático, arrogante, gentil, questionador, sarcástico, bondoso, rigoroso, rebelde, racional, emocional, aventureiro, prudente, ético, manipulável, resiliente, confuso, audacioso, obediente, altruísta, egoísta, controlador, precavido, trabalhador, preguiçoso, perfeccionista, desorganizado, criativo, pragmático, rígido, metódico, procrastinador, desconfiado, dominador, generoso, esperançoso, focado, inovador, conservador, inconsequente, responsável, ansioso, mediador, inflexível, empático, exigente, submisso, autoritário.
        Contexto: \n {texto}
    """),
)

trait_chain = trait_prompt | llm_model | StrOutputParser()

#### Definição da pipeline

In [ ]:
fetch_metadata_runnable = RunnableLambda(
    lambda inp: extract_book_metadata(inp["book_id"])
)

enrich_with_characters_runnable = RunnableLambda(
    lambda book: enrich_books_with_characters(book)
)

book_metadata_pipeline = fetch_metadata_runnable | enrich_with_characters_runnable
full_book_pipeline = book_metadata_pipeline | RunnableLambda(process_book_characters)

#### Execução da pipeline

In [ ]:
if __name__ == "__main__":

    path = 'livros_ptbr.csv'
    if os.path.exists(path):
        df = pd.read_csv(path)
        book_ids = df['id'][51:]
    else:
        print("Arquivo não encontrado, usando um ID de teste.")
        book_ids = [69187]

    for book_id in book_ids:
        try:
            pipeline_input = {"book_id": book_id}
            results = full_book_pipeline.invoke(pipeline_input)

            for index, item in enumerate(results):
                print(f"\n\nResultado {index + 1}:")
                print(item)

                sleep(5)
        except Exception as error:
            print("Falha ao executar extração de dados para o livro", str(error))



Processando Graziella para o livro A capital federal (impressões de um sertanejo)


Resultado 1:
{'character_name': 'Graziella', 'character_context': CharacterLLMInput(book=BookMetadataOutput(title='A capital federal (impressões de um sertanejo)', authors=[Person(name='Coelho Netto, Henrique', birth_year=1864, death_year=1934)], summaries=['"A Capital Federal (Impressões de um Sertanejo)" by Henrique Coelho Netto is a novel written in the early 20th century. The work likely explores the contrasts between rural life and urban experiences in Brazil, highlighting the thoughts and reflections of the main character, Anselmo, as he transitions from the countryside to the busy city. The narrative promises insights into the socio-political landscape of the time, framed through Anselmo\'s personal journey and encounters.  The opening of the novel introduces us to Anselmo, who is traveling to the city of Rio de Janeiro for the first time. He reflects on his lengthy train journey, appreciating 

In [ ]:
pd.read_csv("characters_dataset.csv").head(20)

,title,authors,summaries,subjects,characters,text_url,character_name,character_resume,snippet,traits
0,A capital federal (impressões de um sertanejo),"[{""name"": ""Coelho Netto, Henrique"", ""birth_yea...","[""\""A Capital Federal (Impressões de um Sertan...","[""Brazilian literature"", ""Romance fiction, Bra...","[""Graziella"", ""Sertanejo""]",https://www.gutenberg.org/ebooks/70121.txt.utf-8,Graziella,"Graziella é uma mulher melancólica, sensível e...","O conservador, com um gesto nobre, rejeitou; e...","[""melancólico"", ""sensível"", ""discreto"", ""tímid..."
1,Contos Phantasticos: segunda edição correcta e...,"[{""name"": ""Braga, Teófilo"", ""birth_year"": 1843...","[""\""Contos Phantasticos\"" by Teófilo Braga is ...","[""Fiction"", ""Short stories, Portuguese""]","[""Contos Phantasticos""]",https://www.gutenberg.org/ebooks/32646.txt.utf-8,Contos Phantasticos,Contos Phantasticos é um trabalho generoso e a...,This ebook is for the use of anyone anywhere i...,"[""generoso"", ""altruísta"", ""responsável"", ""ético""]"
2,Contos Phantasticos: segunda edição correcta e...,"[{""name"": ""Braga, Teófilo"", ""birth_year"": 1843...","[""\""Contos Phantasticos\"" by Teófilo Braga is ...","[""Fiction"", ""Short stories, Portuguese""]","[""Contos Phantasticos""]",https://www.gutenberg.org/ebooks/32646.txt.utf-8,Contos Phantasticos,Contos Phantasticos é um trabalho generoso e a...,Updated editions will replace the previous one...,"[""perfeccionista"", ""metódico"", ""responsável"", ..."
3,Flirts,"[{""name"": ""Vasconcellos, Henrique de"", ""birth_...","[""\""Flirts\"" by Henrique de Vasconcellos is a ...","[""Fiction""]","[""Maria do Carmo""]",https://www.gutenberg.org/ebooks/35073.txt.utf-8,Maria do Carmo,"Maria do Carmo é uma mulher de trinta anos, ca...","Maria do Carmo, curvada sobre a meza, folheia ...","[""vaidoso"", ""frio""]"
4,Flirts,"[{""name"": ""Vasconcellos, Henrique de"", ""birth_...","[""\""Flirts\"" by Henrique de Vasconcellos is a ...","[""Fiction""]","[""Maria do Carmo""]",https://www.gutenberg.org/ebooks/35073.txt.utf-8,Maria do Carmo,"Maria do Carmo é uma mulher de trinta anos, ca...","MARIA DO CARMO--trinta annos, com dez de casam...","[""vaidoso"", ""curioso"", ""misterioso"", ""manipula..."
5,A capital federal (impressões de um sertanejo),"[{""name"": ""Coelho Netto, Henrique"", ""birth_yea...","[""\""A Capital Federal (Impressões de um Sertan...","[""Brazilian literature"", ""Romance fiction, Bra...","[""Graziella"", ""Sertanejo""]",https://www.gutenberg.org/ebooks/70121.txt.utf-8,Graziella,"Graziella é uma mulher melancólica, sensível e...","O conservador, com um gesto nobre, rejeitou; e...","[""melancólico"", ""sensível"", ""discreto"", ""tímid..."
6,Contos Phantasticos: segunda edição correcta e...,"[{""name"": ""Braga, Teófilo"", ""birth_year"": 1843...","[""\""Contos Phantasticos\"" by Teófilo Braga is ...","[""Fiction"", ""Short stories, Portuguese""]","[""Contos Phantasticos""]",https://www.gutenberg.org/ebooks/32646.txt.utf-8,Contos Phantasticos,Contos Phantasticos é um trabalho generoso e a...,This ebook is for the use of anyone anywhere i...,"[""generoso"", ""altruísta"", ""responsável"", ""ético""]"
7,Contos Phantasticos: segunda edição correcta e...,"[{""name"": ""Braga, Teófilo"", ""birth_year"": 1843...","[""\""Contos Phantasticos\"" by Teófilo Braga is ...","[""Fiction"", ""Short stories, Portuguese""]","[""Contos Phantasticos""]",https://www.gutenberg.org/ebooks/32646.txt.utf-8,Contos Phantasticos,Contos Phantasticos é um trabalho generoso e a...,Updated editions will replace the previous one...,"[""perfeccionista"", ""metódico"", ""responsável"", ..."
8,Flirts,"[{""name"": ""Vasconcellos, Henrique de"", ""birth_...","[""\""Flirts\"" by Henrique de Vasconcellos is a ...","[""Fiction""]","[""Maria do Carmo""]",https://www.gutenberg.org/ebooks/35073.txt.utf-8,Maria do Carmo,"Maria do Carmo é uma mulher de trinta anos, ca...","Maria do Carmo, curvada sobre a meza, folheia ...","[""vaidoso"", ""f